In [ ]:
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
from etl import extract_data, transform_data, generate_report
from data_validation import validate_data

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2023, 1, 1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'daily_operational_monitoring',
    default_args=default_args,
    description='Daily operational efficiency monitoring',
    schedule_interval='@daily',
    catchup=False,
)

def perform_etl():
    df = extract_data()
    df = transform_data(df)
    validation_errors = validate_data(df)
    
    if validation_errors:
        raise ValueError(f"Data validation failed: {validation_errors}")
    
    generate_report(df)
    
    # Check for underperforming regions
    underperforming = df[df['is_underperforming']]
    if not underperforming.empty:
        # In a real scenario, you would send an alert here
        print("ALERT: Underperforming regions detected!")
        print(underperforming)

etl_task = PythonOperator(
    task_id='perform_etl',
    python_callable=perform_etl,
    dag=dag,
)

etl_task